In [1]:
import os
import sys
import tensorflow as tf

reader_path = "../" 
sys.path.append(os.path.abspath(reader_path))
import reader

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
class Config(object):
    # The PTB dataset comes from Tomas Mikolov's webpage:
    # http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz
    data_path = "/tmp/ptb_data"
    init_scale = 0.05
    hidden_size = 256
    num_layers = 2
    vocab_size = 10000
    learning_rate = 1.0
    batch_size = 16
    num_steps = 32
#     eval_batch_size = 1
#     eval_num_steps = 1
    num_epoch = 2
    keep_prob = 0.5
    max_grad_norm = 5

def get_config():
    return Config

In [3]:
class PTBModel(object):
    def __init__(self, is_train, config, data):
        self.batch_size = config.batch_size
        self.num_steps = config.num_steps
        self.epoch_size = ((len(data) // self.batch_size) - 1) // self.num_steps
        self.input_data, self.targets = reader.ptb_producer(
            data, self.batch_size, self.num_steps)
        vocab_size = config.vocab_size
        hidden_size = config.hidden_size
        
        embedding = tf.get_variable("embedding", [vocab_size, hidden_size])
        inputs = tf.nn.embedding_lookup(embedding, self.input_data) # batch_size*num_steps*hidden_size
        if is_train and config.keep_prob<1:
            inputs = tf.nn.dropout(inputs, config.keep_prob)
        
        output, state = self._build_lstm_graph(inputs, config, is_train)  
        
        softmax_w = tf.get_variable("softmax_w", [hidden_size, vocab_size])
        softmax_b = tf.get_variable("softmax_b", [vocab_size])
        logits = tf.nn.xw_plus_b(output, softmax_w, softmax_b)
        # Reshape logits to be a 3-D tensor for sequence loss
        logits = tf.reshape(logits, [self.batch_size, self.num_steps, vocab_size])

        loss = tf.contrib.seq2seq.sequence_loss(
            logits,
            self.targets,
            tf.ones([self.batch_size, self.num_steps], dtype = tf.float32),
            average_across_batch=True)
        self.cost = tf.reduce_sum(loss)
        self.final_state = state
        
        if not is_train:
            return
        
        # 控制梯度膨胀
        train_vars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(
            tf.gradients(self.cost, train_vars), config.max_grad_norm)
        optimizer = tf.train.GradientDescentOptimizer(config.learning_rate)
        self.train_op = optimizer.apply_gradients(
            zip(grads, train_vars))
    
    
    def _build_lstm_graph(self, inputs, config, is_train):
        lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(config.hidden_size)
        # Use deep RNN with dropout
        if is_train and config.keep_prob < 1:
            lstm_cell = tf.nn.rnn_cell.DropoutWrapper(
                lstm_cell, output_keep_prob=config.keep_prob)
        cell = tf.nn.rnn_cell.MultiRNNCell([lstm_cell] * config.num_layers)
        self.initial_state = cell.zero_state(config.batch_size, tf.float32)
        state = self.initial_state
        outputs = []
        with tf.variable_scope("RNN", reuse=tf.AUTO_REUSE):
            for time_step in range(self.num_steps):
#                 if time_step > 0: tf.get_variable_scope().reuse_variables()
                output, state = cell(inputs[:, time_step, :], state)
                outputs.append(output)        
        output = tf.reshape(tf.concat(outputs, 1), [-1, config.hidden_size])
        return output, state


In [4]:
def run_epoch(sess, model, train_op=tf.no_op()):
    tota_costs = 0.0
    iters = 0
    state = sess.run(model.initial_state)
    for step in range(model.epoch_size):
        print(step)
        cost, state, _ = sess.run([model.cost, model.final_state, train_op],
                                  feed_dict={model.initial_state: state})
        print(cost)
        total_costs += cost
        iters += model.num_steps
        
        if step % (model.epoch_size // 10) == 0:
            print("Perplexity at step %d is %.3f" % (step, np.exp(total_costs/iters)))
    return np.exp(total_costs/iters)

In [5]:
def main(_):
    config = get_config()
    train_data, valid_data, test_data, _ = reader.ptb_raw_data(config.data_path)
    initializer = tf.random_uniform_initializer(-config.init_scale,
                                                config.init_scale)
    with tf.name_scope("train"):
        with tf.variable_scope("model", reuse=None, initializer=initializer):
            train_model = PTBModel(is_train=True, config=config, data=train_data)
        tf.summary.scalar("Training Loss", train_model.cost)
#         tf.summary.scalar("Learning Rate", model.learning_rate)
    
    with tf.name_scope("validation"):
        with tf.variable_scope("model", reuse=True, initializer=initializer):
            valid_model = PTBModel(is_train=False, config=config, data=valid_data)
        tf.summary.scalar("Validation Loss", valid_model.cost)
    
    with tf.name_scope("test"):
        with tf.variable_scope("model", reuse=True, initializer=initializer):
            test_model = PTBModel(is_train=False, config=config, data=test_data)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(config.num_epoch):
            train_perplexity = run_epoch(sess, train_model, train_model.train_op)
            print("Epoch: %d Train Perplexity: %.3f" % (i + 1, train_perplexity))
#             valid_perplexity = run_epoch(sess, valid_model)
#             print("Epoch: %d Valid Perplexity: %.3f" % (i + 1, valid_perplexity))
#         test_perplexity = run_epoch(sess, test_model)
#         print("Test Perplexity: %.3f" % test_perplexity)

In [ ]:
if __name__=="__main__":
    tf.app.run()

W0322 14:43:34.011439 140447397373696 deprecation.py:323] From /home/johnzyh/Jupyter/deep learning/lstm/reader.py:120: range_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0322 14:43:34.015135 140447397373696 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/training/input.py:320: input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0322 14:43:34.015943 140447397373696 depreca


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



W0322 14:43:37.825005 140447397373696 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/array_grad.py:425: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
I0322 14:43:37.856920 140447397373696 summary_op_util.py:92] Summary name Training Loss is illegal; using Training_Loss instead.
I0322 14:43:38.739596 140447397373696 summary_op_util.py:92] Summary name Validation Loss is illegal; using Validation_Loss instead.


0
